Chatbot For Medical Advisory using Deep Learning

NLTK stands for Natural Language ToolKit. It has many algorithms available for text tokenisation, stemming, classification and clustering etc. It is also helpful Python programs which work with human language data for applying in statistical natural language processing.
WordNetLemmatizer contains two words - Wordnet (It is freely available lexical database for the English language which aim to establish structural semantic relationships between words.) and Lemmatizing (It is a process of grouping together the different inflected forms of a word, so that they can be analysed together.)

In [143]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\as041\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\as041\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Keras is an open source software library that provides a python interface for Artificial Neural Networks. It runs on top of tensorflow. It helps in building Machine Learning Models. Keras.models helps in importing required model for your project. Keras.layers help in adding hidden layers to our model. Keras.optimisers helps in importing correct optimiztion criteria for our model.

In [144]:
from keras.layers import Dense, Dropout
from keras.models import Sequential


Numpy is a python library, helps in performing various mathematical functions on arrays.
JSON is a standard file format and data interchange format that uses human readable text to store and interchange.

In [145]:
import numpy
import random
import json


Intents.JSON is the Project Data File. It contains list of various intents['tags', 'pattern', 'responses', 'context'] in which intents['tags] defines about the criteria in which a user question falls, intents[pattern'] defines questions (same questions can be asked in many ways) and intents['pattern'] defines about the reponses the user will get on asking the question present in intents['pattern]. Intents['context] tells about relation of one intent to other intent.

In [146]:
# Loading intents in the workbook as a training dataset
intents = {}
with open("Chatbot_intents.json") as f:
    intents = json.load(f)
print("Intents Loaded as Dataset")


Intents Loaded as Dataset


Words is an array which conatain total number of unique words used in the formation of intents['patterns'] in Intents.JSON.
Classes is an array which contain the intents['tags'] under which the user question patterns criteria is formed.
Documents is an array which contain intents['pattern'] with intents['tags'], whcih is helpful in generating input data.

In [147]:
words = []
classes = []
documents = []


In [148]:
for intent in intents['intents']:
    # Creating a list of intents['tags'] to define various criteria under which user questions can fall
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
        # Tokenizing every question pattern present in intent['patterns'] in intents['intents] of intents.JSON
        w = nltk.word_tokenize(pattern)
        # Extending every tokezised word in Words to train the model
        words.extend(w)
        # Documents are appended with tokenized words with their intents['tags'] to create a corpus of data for model 
        documents.append((w, intent['tag']))


In [149]:
# Words array contain every tokenized words in the intents['pattern'] of intents.JSON, which account for
# many duplicate words and unwanted words in the Words[]. Also, every word in the Words[] array should be
# in lowercase alphabetical form for better understanding and comparison.
# WordNetLemmatizer is used to remove duplicacy and unwanted words from the Words[].
ignore_words = ['?', '!', ',']
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]


In [150]:
# Documents[] tells about the total number of combination between the intents['tags'] and intents['pattern'] has been formed
print(len(documents), "Documents")

# Sorting the classes in Alphabetical Order
classes = sorted(list(set(classes)))
# Length of Classes[] will tell about the number of intents our chatbot would be using
# while comparing or responding back to the user
print(len(classes), "Classes")

# set() will find out unique words from the Words[] and sorted() will sort them into alphabetical order
words = sorted(list(set(words)))
# Words[] conatins unique words used in the formation of the the pattern of questions  
print(len(words), "Unique Lemmatized Words")


328 Documents
71 Classes
146 Unique Lemmatized Words


In [151]:
# with open("words.json", "w") as f:
#     json.dump(words, f)
#     print("Words Done")
    
# with open("classes.json", "w") as f:
#     json.dump(classes, f)
#     print("Classes Done")

# with open("documents.json", "w") as f:
#     json.dump(documents, f)
#     print("Dosuments Done")


In [152]:
# Creating training array to store training data
training = []

# Creating training corpus for the model
for doc in documents:
    # Taking out tokenised word list stored in documents at 0th position
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Defining an array to store the words in 0 & 1 as 1 represents words present in Unique Lemmatize Words[]
    bag = []
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Creating an array "output_row" having length equal to Classes[]   
    output_row = list([0] * len(classes))
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Random shuffle the training list and convert it into a numpy array
random.shuffle(training)
training = numpy.array(training, dtype=object)

# Creating training and testing corpus in which X - contains patterns from intents.JSON & Y - contains intents from intents.JSON
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training Data Created")


Training Data Created


Creating Model For Chatbot and training the model with different intent and hyperparameter

For model, we choose tensorflow.keras.models.Sequential(). We added 3 tensorflow.keras.layers.Dense() layers and 2 tensorflow.keras.layers.Dropout() layers to create the model. First Dense layers named "layer_1" contain 128 neurons with input shape equal to length(train_x[0]) (or length(words[])), having activation function "relu". Second Dense layer named "layer_2" contain 64 neurons with input equal to the output of layer_1, having activation function "relu". Third Dense layer is named "layer_3" whose number of neurons equal to len(train_y[0]) (or len(classes[])) with activation function as "softmax". Two Dropouts layers name "dropout_1" and dropout_2" are added to reduce overfitting and overall loss of the model.

For model compilation, tensorflow.keras.optimizers.RMSprop is used which gives best accuracy with minimal "categorical_crossentropy" loss. After compilation, training of model is done. For training in model.fit, x - train_x, y - train_y is used, with batch_size of 5, total number of epochs used are 200.

In [153]:
# Creating the model
model = Sequential()
# Adding first Dense layer
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu', name = "layer_1"))
# Adding first Dropout layer
model.add(Dropout(0.5, name = "dropout_1"))
# Adding second Dense layer
model.add(Dense(128, activation='relu', name = "layer_2"))
# Adding second Dropout layer
model.add(Dropout(0.5, name = "dropout_2"))
# Adding third Dense layer
model.add(Dense(len(train_y[0]), activation='softmax', name = "layer_3"))

# Tells about the summary of the model
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

# Training the model
model.fit(numpy.array(train_x), numpy.array(train_y),
                  epochs=200, batch_size=20)


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 256)               37632     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 layer_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 layer_3 (Dense)             (None, 71)                9159      
                                                                 
Total params: 79,687
Trainable params: 79,687
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
17/17 [==============================

Below cell tells about the "Total Model Loss" and "Total Model Accuracy". Using "RMSprop" optimizers, which gives the least loss and highest accuracy.

In [154]:
# This cell tells about the evaluation metrics of our model
score = model.evaluate(numpy.array(train_x), numpy.array(train_y), verbose=True)

# It prints about the Model Loss
print("Model Loss: " , score[0])

# It prints about the Model Accuracy
print("Model Accuracy: " , score[1])


11/11 [==============================] - 0s 2ms/step - loss: 1.0608e-04 - accuracy: 1.0000
Model Loss:  0.00010608316370053217
Model Accuracy:  1.0


Program for getting responses from the Chatbot

In [155]:
# This function helps in creating a bag[] which can be used as a test data for our model
# "sentence" arguments contain the question pattern asked by the user.
# "words" refers to the words[] which contain unique lemmatize words from intents.JSON  
def sentence_in_words(sentence):
    # "sentence" gets tokenize and lowered in order to match the holdings of words[]
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence)]
    
    # Creating a bag[] whose length equal to words[] in order to compare it with the train_x[]
    bag = [0] * len(words)
    
    # Substituting 1 at places where "sentence_words" is found in the words[] in order to
    # generate input of the model
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    
    return (numpy.array(bag))


In [156]:
# This function helps in predicting the intent i.e. the intents['tag'] under which question
# has highest falling probability and return the intent name with probability
# "sentence" argument contains the question pattern asked by the user
# "model" refers to the model created for the project
def predicting_intent(sentence):
    # bag[] containing the compared data from the words[] obtained from the sentence_in_words () 
    bag = sentence_in_words(sentence)
    
    # predicting the intents['tag'] under which the bag[] may fall
    res = model.predict(numpy.array([bag]))[0]
    
    # [i, r] where i contain the index of classes[], of which intent['tags'] user question belongs
    # and r conatins the predicted probability of user pattern falling into the intents 
    # It is possible to have more than one intent with same pattern, then the intent with highest probability will be considered
    # results.sort() sorts the result in the descending order on the basis of the probability predicted
    results = [[i, r] for i, r in enumerate(res)]
    results.sort(key=lambda x: x[1], reverse=True)
    
    # finding the intents['tags'] with the help of classes[] of the result[0] which has the 
    # highest probability (from whcih the user question pattern may belong), if probability is not found, then
    # "do not understand question" reaponse get geenrated
    if (results[0][0] < 0.50):
        results = [{"intent": 'i do not understand', "probability": str(results[0][1])}]
    else:
        results = [{"intent": classes[results[0][0]], "probability": str(results[0][1])}]
    
    return results


In [157]:
# This function generates the actual response to be given by the chatbot, for the user's question pattern
def get_Response(ints):
    # ints contain the result from the predicting_intent() which has both intent name and its probability
    # tag variable gets the intent name from the ints
    tag = ints[0]['intent']
    
    # comparing the tag name with the intents['tags'] of intents.JSON to generate a random response
    for i in intents['intents']:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            return result
    

In [158]:
# This is a function to call other function
def chatbot_response(msg):
    # This function returns with the intent['tags'] with highest probability 
    # under which the "msg" has fallen
    ints = predicting_intent(msg)
    
    # This function returns with one of the random responses from the intents['responses']
    # related to the user question pattern
    response = get_Response(ints)
    
    return response


In [161]:
# Ask Your Ques Here
ques = "Hello"
print(chatbot_response(ques))

Hi there, How can i help?
